In [68]:
import pandas as pd
import numpy as np

In [69]:
df=pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [70]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [71]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [72]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=scaler.fit_transform(X)
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [73]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [74]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [75]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

c:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [76]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

In [77]:
model.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32)

Epoch 1/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5468 - loss: 0.7006 - val_accuracy: 0.5779 - val_loss: 0.6978
Epoch 2/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6365 - loss: 0.6418 - val_accuracy: 0.6234 - val_loss: 0.6429
Epoch 3/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6869 - loss: 0.5886 - val_accuracy: 0.6364 - val_loss: 0.6078
Epoch 4/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7037 - loss: 0.5684 - val_accuracy: 0.6558 - val_loss: 0.5827
Epoch 5/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7210 - loss: 0.5461 - val_accuracy: 0.6558 - val_loss: 0.5648
Epoch 6/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7441 - loss: 0.5257 - val_accuracy: 0.6688 - val_loss: 0.5509
Epoch 7/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7479 - loss: 0.5182 - val_accuracy: 0.6688 - val_loss: 0.5409
Epoch 8/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7626 - loss: 0.4967 - val_accuracy: 0.6883 - val_loss

In [78]:
# Selecting the appropriate optimizer
import keras_tuner as kt

In [79]:
def build_model(hp): #hp-> hyperparameter object you get it int the function itself
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice("optimizer",values=['adam','sgd','rmsprop','adadelta']) # optimizer is simply a name can be abc # individual models will be a different optimizer each time
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=["accuracy"]) 
    return model

In [80]:
# tuner object
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [81]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [82]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [83]:
model=tuner.get_best_models(num_models=1)[0] # to get the best model
model.summary()

c:\Users\HP\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test),batch_size=32) # initial_epoch=5 as the model has been trained for 5 epochs already and we want to retain that information

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7359 - loss: 0.6024 - val_accuracy: 0.7792 - val_loss: 0.5657
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7411 - loss: 0.5754 - val_accuracy: 0.7662 - val_loss: 0.5455
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7727 - loss: 0.5321 - val_accuracy: 0.7727 - val_loss: 0.5289
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7605 - loss: 0.5305 - val_accuracy: 0.7792 - val_loss: 0.5191
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7445 - loss: 0.5217 - val_accuracy: 0.7792 - val_loss: 0.5099
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7575 - loss: 0.5142 - val_accuracy: 0.7792 - val_loss: 0.5047
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7632 - loss: 0.4789 - val_accuracy: 0.7792 - val_loss: 0.5009
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7994 - loss: 0.4603 - val_accuracy: 0.792

In [85]:
# selecting the correct number of neurons

In [86]:
def build_model(hp):

    model=Sequential()
    
    units=hp.Int("units",min_value=8,max_value=128,step=8) # lowerLimit=8, upperLimit=128 #multiple anns with increasing number of neurons will be built and the the best performing one will be retained

    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(units=1,activation='sigmoid'))

    model.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')

    return model

In [87]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5,
                      directory="myDir") # to maximize val_accuracy

Reloading Tuner from myDir\untitled_project\tuner0.json


In [88]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [89]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [90]:
model=tuner.get_best_models(num_models=1)[0] # to get the best model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 120)            │         1,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,201 (4.69 KB)

 Trainable params: 1,201 (4.69 KB)

 Non-trainable params: 0 (0.00 B)

In [91]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test),batch_size=32)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7811 - loss: 0.4763 - val_accuracy: 0.7857 - val_loss: 0.4874
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7548 - loss: 0.4826 - val_accuracy: 0.7857 - val_loss: 0.4902
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7907 - loss: 0.4490 - val_accuracy: 0.7792 - val_loss: 0.4880
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7704 - loss: 0.4562 - val_accuracy: 0.7857 - val_loss: 0.4899
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7980 - loss: 0.4359 - val_accuracy: 0.7662 - val_loss: 0.4933
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7823 - loss: 0.4508 - val_accuracy: 0.7597 - val_loss: 0.4958
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7969 - loss: 0.4314 - val_accuracy: 0.7662 - val_loss: 0.4948
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7834 - loss: 0.4531 - val_accuracy: 0.759

In [92]:
# seleting the number of layers

In [93]:
def build_model(hp):

    model=Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model


In [94]:
tuner=kt.RandomSearch(build_model,
                      objective="val_accuracy",
                      max_trials=3,
                      directory="myDir",
                      project_name="num_layers")

Reloading Tuner from myDir\num_layers\tuner0.json


In [95]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [96]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [97]:
model=tuner.get_best_models(num_models=1)[0] # to get the best model
model.summary()

c:\Users\HP\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,513 (146.54 KB)

 Trainable params: 37,513 (146.54 KB)

 Non-trainable params: 0 (0.00 B)

In [98]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test),batch_size=32)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7757 - loss: 0.4699 - val_accuracy: 0.7468 - val_loss: 0.5207
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7578 - loss: 0.4529 - val_accuracy: 0.7727 - val_loss: 0.5132
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8224 - loss: 0.3979 - val_accuracy: 0.7792 - val_loss: 0.5333
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8221 - loss: 0.4028 - val_accuracy: 0.7532 - val_loss: 0.5380
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8373 - loss: 0.3844 - val_accuracy: 0.7727 - val_loss: 0.5662
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8648 - loss: 0.3402 - val_accuracy: 0.6948 - val_loss: 0.5943
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8633 - loss: 0.3423 - val_accuracy: 0.7468 - val_loss: 0.5734
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8586 - loss: 0.3422 - val_accuracy: 0.733

In [99]:
# A model with all the hyperparameters tuned 

In [100]:
def build_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int("num_layers",min_value=1,max_value=10)):

        if counter==0:
            # if it is the first layer
            model.add(
                Dense(
                    hp.Int('units'+str(i),min_value=8,max_value=128,step=8), #'units'+str(i) as every loopiteration is expected to have a different name
                    activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                    input_dim=8
                )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4])))
        else:
            model.add(
                Dense(
                    hp.Int('units'+str(i),min_value=8,max_value=128,step=8), #'units'+str(i) as every loopiteration is expected to have a different name
                    activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
                )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4])))
            
        counter+=1
    
    model.add((Dense(1,activation='sigmoid'))) # last layer

    model.compile(optimizer=hp.Choice('optimizer',values=['adam','adadelta','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [101]:
tuner=kt.RandomSearch(build_model,
                      objective="val_accuracy",
                      max_trials=3,
                      directory="myDir",
                      project_name="tuned_model")

In [102]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 00m 14s


In [103]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units0': 96,
 'activation0': 'tanh',
 'dropout0': 0.3,
 'optimizer': 'adam',
 'units1': 112,
 'activation1': 'tanh',
 'dropout1': 0.4,
 'units2': 128,
 'activation2': 'sigmoid',
 'dropout2': 0.1,
 'units3': 72,
 'activation3': 'sigmoid',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 24,
 'activation5': 'tanh',
 'dropout5': 0.3,
 'units6': 16,
 'activation6': 'relu',
 'dropout6': 0.2,
 'units7': 104,
 'activation7': 'tanh',
 'dropout7': 0.2,
 'units8': 112,
 'activation8': 'tanh',
 'dropout8': 0.4,
 'units9': 112,
 'activation9': 'sigmoid',
 'dropout9': 0.1}

In [104]:
model=tuner.get_best_models(num_models=1)[0] # to get the best model
model.summary()

c:\Users\HP\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 96)             │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 112)            │        10,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        14,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 72)             │         9,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 72)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 24)             │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 104)            │         1,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 104)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 112)            │        11,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           113 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,321 (196.57 KB)

 Trainable params: 50,321 (196.57 KB)

 Non-trainable params: 0 (0.00 B)

In [105]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test),batch_size=32)

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.7301 - loss: 0.5286 - val_accuracy: 0.7013 - val_loss: 0.5515
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7655 - loss: 0.4887 - val_accuracy: 0.6623 - val_loss: 0.5557
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7197 - loss: 0.5370 - val_accuracy: 0.6948 - val_loss: 0.5926
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7718 - loss: 0.5160 - val_accuracy: 0.6818 - val_loss: 0.5683
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7614 - loss: 0.5073 - val_accuracy: 0.6883 - val_loss: 0.5536
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7193 - loss: 0.5564 - val_accuracy: 0.7013 - val_loss: 0.5628
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7565 - loss: 0.4950 - val_accuracy: 0.6753 - val_loss: 0.5637
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7348 - loss: 0.5120 - val_accuracy: 0.688